In [1]:
from scipy.special import comb
import itertools
import numpy as np
import random

In [2]:
class MOEAD:
    # m … 目的数
    # H … 分解パラメータ
    # N … 重みベクトルと解集団サイズ
    # T … 近傍サイズ
    # f … 関数集団
    def __init__(self, m, H, T, fs):
        self.m = m
        self.H = H
        self.N = comb(self.H + self.m - 1, self.m - 1, exact=True)
        self.T = T
        self.fs = fs
        print("m:%d, H:%d, N:%d, T:%d" % (self.m, self.H, self.N, self.T))
        
    # 初期化フェーズ
    def init_phase(self):
        # 重みベクトル群の生成
        self.L = []
        for combo in itertools.combinations(range(self.H + self.m - 1), self.m - 1):
            lamb = [combo[0]]
            for i in range(1, self.m - 1):
                lamb.append(combo[i] - combo[i - 1] - 1)
            lamb.append(self.H + self.m - 2 - combo[self.m - 2])
            
            self.L.append(np.array(list(map(lambda x: x / self.H, lamb))))
        if len(self.L) != self.N:
            print("重みベクトルの生成数が正しくありません N:%d != L:%d" % (self.N, self.L))
            
        # 近傍重みベクトル群を見つける
        self.B = [None for _ in range(len(self.L))]
        for i, lamb_i in enumerate(self.L):
            distances = {idx : np.linalg.norm(lamb_i - lamb_j) for idx, lamb_j in enumerate(self.L)}
            sorted_dis = sorted(distances.items(), key = lambda x : x[1])
            self.B[i] = sorted([sorted_dis[k][0] for k in range(self.T)])
            
        # 初期集団生成
        init_rand_min = 1.0
        init_rand_max = 2.0
        self.x = np.array([lamb * random.uniform(init_rand_min, init_rand_max) for lamb in self.L])
        
        # 理想点初期化
        self.z = np.array([0 for _ in range(self.m)])
            
    def solution_search_phase(self, generation):
        for g in range(generation):
            for i in range(self.N):
               # 親選択
                p, q = random.sample(self.B[i], 2)
                x_p, x_q = self.x[p], self.x[q]
                
                #交差,突然変異
                y = np.array()
                
                # 理想点更新
                for j in range(self.m):
                    if y[j] < self.z[j]:
                        self.z[j] = y[j]
                
                # 解集団の更新
                for j in self.B[i]:
                    lamb = self.L[j]
                    g_x = max([f(self.x[j]) / lamb[k] for k, f in enumerate(self.fs)])
                    g_y = max([f(y) / lamb[k] for k, f in enumerate(self.fs)])
                    if g_y < g_x:
                        self.x[j] = y
                        
        return self.x        

In [6]:
moead = MOEAD(2,8,3)

m:2, H:8, N:9, T:3


In [7]:
moead.init_phase()

In [12]:
a = np.array([0, 1, 3])
print(list(a))

[0, 1, 3]
